# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Оглавление

[1. Описание данных](#1)\
[2. Импорт модулей](#2)\
[3. Функции](#3)\
[4. Загрузка данных](#4)\
[5. Предобработка данных](#5)\
&emsp;[5.1. Обработка пропусков](#5-1)\
&emsp;[5.2. Замена типа данных](#5-2)\
&emsp;[5.3. Обработка дубликатов](#5-3)\
&emsp;[5.4. Лемматизация](#5-4)\
&emsp;[5.5. Категоризация](#5-5)\
[6. Выводы](#6)\
&emsp;[6.1. Зависимость между наличием детей и возвратом кредита в срок](#6-1)\
&emsp;[6.2. Зависимость между семейным положением и возвратом кредита в срок](#6-2)\
&emsp;[6.3. Зависимость между уровнем дохода и возвратом кредита в срок](#6-3)\
&emsp;[6.4. Влияние разных целей кредита на его возврат в срок](#6-4)\
[7. Итог](#7)

<a id='1'></a>
## 1. Описание данных

* *children* — количество детей в семье
* *days_employed* — общий трудовой стаж в днях
* *dob_years* — возраст клиента в годах
* *education* — уровень образования клиента
* *education_id* — идентификатор уровня образования
* *family_status* — семейное положение
* *family_status_id* — идентификатор семейного положения
* *gender* — пол клиента
* *income_type* — тип занятости
* *debt* — имел ли задолженность по возврату кредитов
* *total_income* — ежемесячный доход
* *purpose* — цель получения кредита

<a id='2'></a>
## 2. Импорт модулей

In [ ]:
!pip install pymystem3 pymorphy2 pymorphy2-dicts-ru

In [ ]:
import requests
import pymorphy2                      # для определения частей речи
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
from urllib.parse import urlencode
from nltk.stem import SnowballStemmer # для стемминга

<a id='3'></a>
## 3. Функции

In [ ]:
LINKS = ["https://yadi.sk/d/asLAt64H9tBe0A"]
PATHS = ['/datasets/data.csv']

In [ ]:
def load_data(paths, links=None):
    """
    Функция принимает пути до локальных файлов с данными и (опционально) ссылки
    для их скачивания.
    
    paths - локальные пути до файлов с данными
    links - ссылки для скачивания (Яндекс.Диск)
    """
    
    data_list = []
    base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
    
    if links != None and not links:
        raise Exception("Error: wrong 'links' value (nmust be not empty list)")
    
    if links:
        if len(paths) != len(links):
            raise Exception("Error: length of 'links' must be equal to length of 'paths'")

        if type(paths).__name__ != 'list' or type(links).__name__ != 'list':
            raise Exception("Error: variables 'links' and 'paths' must be 'list' type")

    for i in range(len(paths)):
        
        try:
            path = paths[i]
            data_list.append(pd.read_csv(path))
            
        except FileNotFoundError:
            # download from yandex disk
            public_key = links[i]
            print("Run load", public_key)
            
            # get download link
            final_url = base_url + urlencode(dict(public_key=public_key))
            response = requests.get(final_url)
            download_url = response.json()['href']
            data = pd.read_csv(download_url)
            data_list.append(pd.DataFrame(data))
            print('Done.')
            
    return data_list

In [ ]:
def clean_lemmas_rating(rating_dict):
    '''
    Функция для удаления предлогов, пробелов, символа конца строки.
    На вход принимаем список кортежей (<count>, <word>):
    count - частота встречаемости слова во тестовой выборке
    word - слово из целей кредита
    '''
    
    result = []
    
    for item in rating_dict.items():
        count, word = item
        ps = morph.parse(word)[0].tag.POS
        
        try:
            # часть речи (part of speech)
            ps = morph.parse(word)[0].tag.POS
        except:
            # проблемы с кодировкой или внезапно другой язык в данных
            # оставляем результат без изменений, не смогли обработать
            print('Error - processing: {} ->'.format(word), rating)
            
        # оставляем только существительные
        if ps == 'NOUN':
            result.append((word, count))
            
    return dict(result)

In [ ]:
def purpose_category(purpose, rating):
    '''
    Функция для определения категорий целей кредита.
    Проходим от наиболее популярного слова в rating до наименее популярного.
    При первом совпадениии возвращаем категорию.
    rating - аргумент по умолчанию
    '''
    
    for category in rating:
        # получаем основу слова категории, так как в целях используются разные падежи
        stemmed_category = russian_stemmer.stem(category)
        
        if stemmed_category in purpose:
            return category

    return 'NaN'

In [ ]:
def get_debt_probability(df, idx, col='gender', val='debt'):
    """
    Функция для создания совдных таблиц по разным параметрам
    """

    df_debt = person_data.groupby(idx)[val].sum().to_frame().reset_index()
    df_all = person_data.groupby(idx)[col].count().to_frame().reset_index()
    df_data = df_debt.merge(df_all,
                            on=idx,
                            how='inner')
    df_data.rename(columns={col:"all"}, inplace=True)

    # создаем столбец с вероятностью долга
    probability = val+'_probability'
    df_data[probability] = (df_data[val] / df_data['all'] * 100).round(1)
    df_data.sort_values(probability, ascending=False, inplace=True)
    
    return df_data

In [ ]:
def income_category(income, val_25, val_50, val_75):
    """
    Функция для разбиения на категории уровню дохода
    income - значения дохода
    """
    
    if income < val_25:
        return 'низкий'
    
    if val_25 <= income <= val_50:
        return 'ниже среднего'
    
    if val_50 < income <= val_75:
        return 'средний'
    
    else:
        return 'высокий'

<a id='4'></a>
## 4. Загрузка данных

In [ ]:
data = load_data(PATHS, links=LINKS)[0]
data.info()

In [ ]:
# переименуем столбцы
data.rename(columns={"dob_years": "age"}, inplace=True)

data.sample(10)

In [ ]:
null_days_employed = data[ data['days_employed'].isnull()].shape[0]
null_total_income = data[ data['total_income'].isnull()].shape[0]

print(null_days_employed, null_total_income)

In [ ]:
data_no_days_employed = data[data['days_employed'].isnull()]
data_no_days_employed['income_type'].value_counts()

Данные представляют включают признаки 21525 объектов. Выявлены следующие особенности:
1. Пропуски в столбцах _days_employed,  total_income_ в одинаковом количестве - 2174.
2. Разнородность описания:
   * столбец _gender_ неинформативно описывает пол заемщика;
   * встречаются одинаковые значения, написанные в разном регистре.
3. Отрицательные значения столбца _days_employed_ .
4. Типы данных столбцов: числовые (целочисленные и дробные), строковые.
4. Столбцы _family_status, purpose_ - категориальные признаки.
5. Столбец _purpose_ содержит значения, часто отличающиеся падежами (с включениями предлогов), но имеющие одинаковый смысл: "на покупку автомобиля", "на покупку своего автомобиля", "приобретение автомобиля".

<a id='5'></a>
## 5. Предобработка данных

<a id='5-1'></a>
### 5.1. Обработка пропусков

In [ ]:
# взаимосвязь пропусков в 'days_employed' и 'total_income' с значением 'income_type'
null_data = data[data.days_employed.isnull() | data.total_income.isnull()][
    ['days_employed','income_type', 'total_income']]

if null_days_employed == null_data.shape[0] and null_total_income == null_data.shape[0]:
    print('Пропуски для total_income и days_employed совпадают:', null_data.shape[0])

In [ ]:
# Количество пропусков 'total_income' для каждого типа занятости
data[data['total_income'].isnull()]['income_type'].value_counts()

In [ ]:
# Количество пропусков 'days_employed' для каждого типа занятости
data[data['days_employed'].isnull()]['income_type'].value_counts()

In [ ]:
# Посмотрим на значения возраста
data.groupby('age')['education'].count().head()

Во многих заявках возраст не указан, поэтому посмотрим, для каких типов занятости встречается нулевой возраст.

In [ ]:
data[data['age'] == 0]['income_type'].value_counts()

Для столбцов **income_type, days_employed, age** встречаются пропуски. Составим отдельную таблицу со средними и медианными значениями по каждому проблемному столбцу.

In [ ]:
# считаем для age
income_type_stat = data.groupby('income_type').agg({'age':['median','mean']})

# считаем для days_employed
income_type_stat = income_type_stat.merge(data.groupby('income_type').agg({'days_employed':['median','mean']}),
                                          on='income_type',
                                          how='inner')

# считаем для total_income
income_type_stat = income_type_stat.merge(data.groupby('income_type').agg({'total_income':['median','mean']}),
                                          on='income_type',
                                          how='inner')

income_type_stat

Заполним пропуски для столбцов **age, days_employed, total_income** медианой по отдельным типам занятости, так как среднее арифметическое расходится с медианным значением.

In [ ]:
data.info()

In [ ]:
for group in ['total_income', 'days_employed']:
    data[group] = (data
                   .groupby('income_type')[group]
                   .transform(lambda x: x.fillna(x.median()))
                  )

data['age'] = (data
               .groupby('income_type')['age']
               .transform(lambda x: x.replace({0: x.median()}))
              )

print('Минимальный возраст в данных:', data['age'].min())
print('Количество пропущенных значений days_employed:', data.days_employed.isnull().sum())
print('Количество пропущенных значений total_income:', data.total_income.isnull().sum())


In [ ]:
# посмотрим на значения данных о поле заемщиков
data['gender'] = data['gender'].replace('M', 'мужской')
data['gender'] = data['gender'].replace('F', 'женский')
print(data['gender'].value_counts())

In [ ]:
popular_gender = data['gender'].value_counts().idxmax()
print('Самый популярный пол:', popular_gender)

Заменим пропуск на значение самого популярного пола заемщиков.

In [ ]:
data.loc[data['gender'] == 'XNA', 'gender'] = popular_gender
print("\n\nДанные 'gender' после обработки:\n")

# print(data['gender'].value_counts())
data.info()

Проверка пропущенных данных о днях занятости __days_employed__ и доходе заемщика __total_income__ показала, что пропуски характерны для одних и тех же объектов. Однако, это не значит, что лица, у которых пропущены значения в этих стобцах, не работают на момент заявки: данные по ошибке могли быть не заполнены или потеряться. Также было обнаружено, что в данных о возрасте в столбце __age__ много нулевых значений (101), что может повлиять на результат в дальнейшем анализе. Поэтому было принято решение заменить пропуски на медианные значения в зависимости от типа занятости.
Также обнаружен неявный (типа Str) единичный пропуск в данных о поле заемщиков и заменен на самый часто втречаемый пол в данных - _женский_.

<a id='5-2'></a>
### 5.2. Замена типа данных

In [ ]:
# преобразование типа данных в столбцах типа float64

for column in ['days_employed', 'age', 'total_income']:
    data[column] = data[column].astype('int')

data.info()

Столбцы __days_employed__, __age__, __total_income__ в выборке были типа _float64_ без округления. Во избежание проблем при поиске дубликатов данных, значения этих столбцов были переведены в целочисленный тип данных.

<a id='5-3'></a>
### 5.3. Обработка дубликатов

In [ ]:
data['education'].value_counts()

In [ ]:
data['family_status'].value_counts()

In [ ]:
data['income_type'].value_counts()

In [ ]:
data['purpose'].value_counts()

Переведем значения категориальных перевенных к нижнему регистру.

In [ ]:
for column in ['education', 'family_status']:
    data[column] = data[column].str.lower()

for column in [ 'education', 'family_status', 'gender', 'income_type', 'purpose']:
    print(data[column].value_counts().index.to_list())

In [ ]:
# сколько дубликатов
data[data.duplicated()].count()

In [ ]:
# Удаляем полностью совпадающие строки и упорядочиваем индексы

data = data.drop_duplicates().reset_index(drop=True)
data[data.duplicated()].count().sum()

В исходной таблице обнаружено 71 полностью идентичных строк, которые могли появиться в силу разных причин:
* несколько заявлений от одного человека
* ошибка оператора
* техническая неисправность

Все дубликаты удалены.

<a id='5-4'></a>
### 5.4. Лемматизация

In [ ]:
m = Mystem()

In [ ]:
# объединяем все цели покупки в один текст для удобства лемматизации
purpose_text = ' '.join(data['purpose'].to_list())

# находим леммы
purpose_lemmas = m.lemmatize(purpose_text)

# считаем количество каждой
lemmas_dict = dict(Counter(purpose_lemmas))

# удаляем пробелы, символ конца строки, некоторые предлоги
for sym in [' ', 'с', 'на', 'для', 'со', '\n']:
    del lemmas_dict[sym]

# и сортируем по убыванию
lemmas_dict_sort = dict(sorted([(value, key) for (key,value) in lemmas_dict.items()], reverse=True))

# 10 самых популярных лемм
lemmas_dict_sort

В десятке популярных такие цели кредита, как _недвижимость, жилье, автомобиль, образование, свадьба, строительство_.

<a id='5-5'></a>
### 5.5. Категоризация данных

In [ ]:
# добавление стажа работы в годах
data['years_employed'] = (data['days_employed']/365).astype('int')

In [ ]:
# разделим данные на 3 таблицы

# 1 - person_data - основные данные
person_data = data[['income_type', 'years_employed', 'age', 'total_income','purpose', \
                    'gender', 'debt', 'children', 'family_status_id', 'education_id']]

# 2 - family_dict - соответствие 'тип семейного положения' - 'идентификатор'
education_dict = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)

# 3 - education_dict - соответствие 'тип образования' - 'идентификатор'
family_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)

# данные таблицы person_data
person_data.head(5)

In [ ]:
# данные таблицы education_dict
education_dict

In [ ]:
# данные таблицы family_dict
family_dict

In [ ]:
# доп. проверка
data2, bins = pd.qcut(person_data['total_income'], 4, retbins=True)
bins

In [ ]:
# значения квантилей
q_25, q_50, q_75 = tuple(
    person_data['total_income'].describe()[['25%','50%', '75%']].to_list())
    
# добавление нового столбца - категория заемщика по уровню дохода
person_data['income_category'] = (
    person_data['total_income']
    .apply(income_category, args=(q_25, q_50, q_75))
)

person_data['income_category'].value_counts()

In [ ]:
morph = pymorphy2.MorphAnalyzer()

lemmas_rating = clean_lemmas_rating(lemmas_dict_sort)
lemmas_rating

Удалим из рейтинга слова, означающие действие, а не цель.

In [ ]:
rm_words = ['покупка', 'получение', 'сделка', 'проведение', 
            'приобретение', 'сдача', 'семья', 'жилой']
for w in rm_words:
    del lemmas_rating[w]
    
lemmas_rating

In [ ]:
russian_stemmer = SnowballStemmer('russian')

# добавление категории заемщика по цели кредита
person_data['purpose_category'] = person_data['purpose'].apply(
    purpose_category, args=(lemmas_rating,))

# заменим 'жилье' -> 'недвижимость'
person_data['purpose_category'] = person_data['purpose_category'].replace('жилье', 'недвижимость')

# посмотрим количество клиентов для каждой категории - если нет NaN, то 
# все прошло успешно
person_data['purpose_category'].value_counts()

Данные были разделены на 3 таблицы:
1. __person_data__    - основные данные заемщика
* __family_dict__    - таблица-словарь для соответствия _'тип семейного положения' - 'идентификатор'_
* __education_dict__ - таблица-словарь _'тип образования' - 'идентификатор'_

Согласно поставленной задаче было решено разбить данные на категории по следующим признакам:
  
* категории __семейного положения__

    В зависимости от семейного положения при заёме учитывается совместный доход супругов. Соответственно, добавлены две категории _"в браке"_ и _"свободные"_.


* категории __уровня дохода__

    Доход в данных был разделен на _"низкий" (< 30000 у.е.), "средний" (30000 - 799999 у.е.), "высокий" (>= 80000 у.е.)_. Пороговые значения выбраны приблизительно, так как неизвестно, какому географическому и экономическому объекту соответствует тестовая выборка.


* категории __целей кредита__

    Процесс определения категорий определялся исходя из результата лемматизации - был получен рейтинг самых часто встречаемых слов в целях, затем проводилась автоматизированная очистка данных фнукцией clean_lemmas_rating() от непрезентабельных для дальнейшего анализа частей речи - всех, кроме существительных. Дополнительно из рейтинга были удалены те существительные, которые не несут смысловой нагрузки категоризации. В результате работы функции purpose_category(), примененной к таблице, данные были разбиты на 4 категории
    
    1. Недвижимость
    * Автомобиль
    * Образование
    * Свадьба

<a id='6'></a>
## 6. Выводы

<a id='6-1'></a>
### 6.1. Зависимость между наличием детей и возвратом кредита в срок

In [ ]:
# так как группы с количеством заемщиков <100 составляют малую часть от других групп (где счет идет на сотни и
# тысячи) и в процентном соотношении дают вбросы, не будем учитывать эти группы при ответе на поставленный вопрос

# считаем вероятность задолжности (debt_probability)
debt_children_data = get_debt_probability(person_data, 'children', col='gender', val='debt')
debt_children_data[debt_children_data["all"] > 100][['children', 'debt_probability']]

Выделен показатель в виде вероятности задолженности:
- у бездетнных людей показатель меньше всего - 7,5%;
- люди с 3 детьми вероятность выше,но не намного - 8,2%;
- заемщики с 1-2 детьми характеризуются близкой вероятностью задолжности - 9,2% и 9,5% соответственно.

При сравнении групп "бездетные", "с одним ребенком" и "с двумя детьми", где группы сравнимы по количеству человек, видно зависимость, что при увеличении количества детей вероятность задолжности увеличивается (нелинейно).

Однако, при рассмотрении дополнительно группы "с 3 детьми" заметно отклонение от сделанного вывода - вероятность снижается с 9,5% до 8,2%. В данном случае группа значительно отличается по численности от остальных - 330 человек против тысячных групп. Соответственно, для более основательного вывода нужно больше данных о людях с количеством детей 3 и более.

<a id='6-2'></a>
### 6.2. Зависимость между семейным положением и возвратом кредита в срок

In [ ]:
# зависимость долгов от семейного положения
debt_family_data = get_debt_probability(person_data, 'family_status_id', col='gender', val='debt')

# добавим расшифровку статуса семейного положения
debt_family_data = debt_family_data.merge(family_dict,
                                         on='family_status_id',
                                         how='inner')
debt_family_data[['family_status', 'debt_probability']]

Выявлена зависимость между семейным положением и возвратом кредита в срок:
* Наименьшая вероятность задолжности у вдовцов/вдов (самая немногочисленная группа) - 6.6%.
* Для людей в разводе и браке вероятность больше - 7.1% и 7.5% соотвественно.
* В гражданском браке вероятность возрастает - 9.3%.
* 9.8% долгов соотвествует категории людей, которые не состоят в отношениях.

<a id='6-3'></a>
### 6.3. Зависимость между уровнем дохода и возвратом кредита в срок

In [ ]:
# считаем вероятность задолжности (debt_probability)
debt_income_data = get_debt_probability(person_data, 'income_category', col='gender', val='debt')
debt_income_data[['income_category', 'debt_probability']]

Выявлена зависимость между уровнем дохода и возвратом кредита в срок:
* наименьшая вероятность задолжности у категорий людей с высоким и низким уровнем дохода - 7.1% и 8%;
* большая вероятность долга у людей со средним уровнем дохода и ниже среднего - 8.5% и 8.8%.

<a id='6-4'></a>
### 6.4. Влияние разных целей кредита на его возврат в срок

In [ ]:
# считаем вероятность задолжности (debt_probability)
debt_purpose_data = get_debt_probability(person_data, 'purpose_category', col='gender', val='debt')
debt_purpose_data[['purpose_category', 'debt_probability']]

Выявлена зависимость между целями кредита и возвратом кредита в срок. Согласно полученным результатам, выявлены следующие вероятности для категорий целей кредита:
* "автомобиль" и "образованиее" имеют наибольший показатель - 9.4% и 9.2% соответственно;
* "свадьба" - 8%;
* "недвижимость" - категория с наименьшей вероятностью  долга - 7.2%.

<a id='7'></a>
## 7. Итог

Данные о заемщиках представлены несколькими признаками, представляющие собой коллчественные и категориальные значения. Значение целей кредита представлены разнородными категориальными переменными, так как при одинаковых смысловых значениях указаны в разных падежах.

При исследовании в данных были обнаружены проблемы:

* __Неинформативные названия столбцов__. Столбец __dob_years__ был переименован на __age__.


* __Пропуски__. В данных найдены явные пропуски, которые могут свидетельствовать о технической проблеме выгрузки или сбора данных. Пропущенные значения о _стаже, общем доходе, возрасте_ заменены на медианные значения в зависимости от типа занятости, а единичный пропуск о _поле_ заемщика был заменен на самый часто встречаемый - _женский_.


* __Разнородность описания данных__. В данных пол заемщиков указан неинформативно - одной латинской буквой. Значения были заменены на значения _"женский"_ и _"мужской"_. Также в данных были указаны идентичные значения в разном регистре. Как результат, все значения приведены к нижнему регистру.


* __Некорректные значения__. В столбцах о стаже и доходе встречаются отрицательные значения. Помимо этого, у некоторых заемщиков указан завышенный стаж работы - более 100 лет. Так как данные о стаже в поставленной задаче не будут учитываться при анализе, значания оставлены без изменений. В доходе отрицательные значения не в большом количестве относительно количества данных, поэтому отрицательные значения дохода также оставлены.


* __Типы данных__. Значения с типом с плавающей запятой (указанные без округления) переведены в целочисленный во избежание ошибок при сравнении количественных переменных и поиске дубликатов.


* __Дубликаты__. В исходной таблице обнаружена совокупность полностью идентичных строк, которые могли появиться в следствие технической проблемы или наличии повторных заемщиков. Так как не поставлена задача исследования повторных заемщиков, полностью совпадающие строки были удалены.


В результате лемматизации были выделены самые популярные цели кредита:
* недвижимость
* жилье
* автомобиль
* образование
* свадьба
* строительство


Для упрощения работы с данными выполнено разделение на 3 таблицы:
1. __person_data__    - основные данные заемщика
* __family_dict__    - таблица-словарь для соответствия _'тип семейного положения' - 'идентификатор'_
* __education_dict__ - таблица-словарь _'тип образования' - 'идентификатор'_


Согласно поставленной задаче было решено разбить данные на категории по следующим признакам:

* категории __уровня дохода__ (низкий, средний, высокий) - пороговые значения выбраны приблизительно, так как
    неизвестно, какому географическому и экономическому объекту соответствуют данные.
* категории __целей кредита__ (недвижимость, жилье, автомобиль, образование, свадьба)


Выявлены следующие зависимости в данных:

* между наличием детей и возвратом кредита в срок - "бездетные", "с одним ребенком" и "с двумя детьми", где группы сравнимы по количеству человек, видно зависимость, что при увеличении количества детей вероятность задолжности нелинейно увеличивается - __7.5%__, __9.2%__, __9.5__% соответственно. Для основательного вывода о зависимости нужно больше данных о людях с количеством детей 3 и более.

* между семейным положением и возвратом кредита в срок. Наименьшая вероятность задолжности у вдовцов/вдов (самая немногочисленная группа) - 6.6%, для людей в разводе и браке вероятность больше - __7.1__% и __7.5__%, а для людей в гражданском браке и свободных вероятность возрастает - __9.3%__ и  __9.8%__.
    
* между уровнем дохода и возвратом кредита в срок. Наименьшая вероятность задолжности у категорий людей с высоким и низким уровнем дохода - __7.1%__ и __8%__. Большая вероятность долга у людей со средним уровнем дохода и ниже среднего - __8.5%__ и __8.8%__.

* между целями кредита и возвратом кредита в срок. Категории "автомобиль" и "образованиее" имеют наибольший показатель - __9.4%__ и __9.2%__, "свадьба" - __8%__, "недвижимость" - категория с наименьшей вероятностью  долга - __7.2%__.